# 30 Day Mortality Identification
Create a patient level dataframe with subject_id, last hospital admission date, date of death, and 30day_mortality columns

In [4]:
import pandas_gbq
from google.oauth2 import service_account
import pandas as pd
import numpy as np

In [9]:
patient_dir = "../data/patientData/"

In [2]:
# apply credentials
credentials = service_account.Credentials.from_service_account_file('../Patient-Similarity-credentials.json')
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "patient-similarity"

# Extract Mortality Data

In [3]:
q= """SELECT A.subject_id, A.last_admission , date(B.DOD) as death_date, DATE_DIFF( date(B.DOD), last_admission, DAY) as time2event
FROM ( SELECT subject_id, MAX(date(ADMITTIME)) as last_admission
      FROM `patient-similarity.mimic.admissions`
      GROUP BY subject_id
      ) as A 
left join `patient-similarity.mimic.patients` as B
using(subject_id)
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts` )"""

mortality = pandas_gbq.read_gbq(q)
mortality

,subject_id,last_admission,death_date,time2event
0,7275,2140-04-12,2140-05-08,26.0
1,357,2199-12-21,2201-08-02,589.0
2,878,2137-11-24,2137-12-11,17.0
3,794,2190-11-29,2191-09-19,294.0
4,1457,2189-01-25,NaT,NaN
...,...,...,...,...
2879,61809,2113-03-02,NaT,NaN
2880,73843,2135-05-31,NaT,NaN
2881,88276,2110-11-17,NaT,NaN
2882,12564,2144-11-19,2145-05-03,165.0


In [7]:
# generate 30-day mortality flag
mortality['mortality30'] = np.where(mortality['time2event'] <= 30, 1, 0)
mortality

,subject_id,last_admission,death_date,time2event,mortality30
0,7275,2140-04-12,2140-05-08,26.0,1
1,357,2199-12-21,2201-08-02,589.0,0
2,878,2137-11-24,2137-12-11,17.0,1
3,794,2190-11-29,2191-09-19,294.0,0
4,1457,2189-01-25,NaT,NaN,0
...,...,...,...,...,...
2879,61809,2113-03-02,NaT,NaN,0
2880,73843,2135-05-31,NaT,NaN,0
2881,88276,2110-11-17,NaT,NaN,0
2882,12564,2144-11-19,2145-05-03,165.0,0


In [8]:
mortality.mortality30.value_counts()

0    2087
1     797
Name: mortality30, dtype: int64

In [10]:
mortality.to_csv(patient_dir + "mortality.csv", index = False)